### Mini-Projeto - Converse com seus Documentos

In [1]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.chroma import Chroma

from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [2]:
caminhos = [
    "files/apostila.pdf",
    "files/LLM.pdf",
    ]

paginas = []
for caminho in caminhos:
    loader = PyPDFLoader(caminho)
    paginas.extend(loader.load())

recur_split = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    separators=["\n\n", "\n", ".", " ", ""]
)

documents = recur_split.split_documents(paginas)

for i, doc in enumerate(documents):
    doc.metadata['source'] = doc.metadata['source'].replace('arquivos/', '')
    doc.metadata['doc_id'] = i

Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 42 0 (offset 0)
Ignoring wrong pointing object 50 0 (offset 0)
Ignoring wrong pointing object 52 0 (offset 0)
Ignoring wrong pointing object 54 0 (offset 0)
Ignoring wrong pointing object 56 0 (offset 0)
Ignoring wrong pointing object 58 0 (offset 0)
Ignoring wrong pointing object 70 0 (offset 0)
Ignoring wrong pointing object 72 0 (offset 0)
Ignoring wrong pointing object 89 0 (offset 0)
Ignoring wrong pointing object 91 0 (offset 0)
Ignoring wrong pointing object 103 0 (offset 0)
Ignoring wrong pointing object 108 0 (offset 0)
Ignoring wrong pointing object 149 0 (offset 0)
Ignoring wrong pointing object 155 0 (offset 0)
Ignoring wrong pointing object 158 0 (offset 0)
Ignoring wrong pointing object 160 0 (offset 0)
Ignoring wrong pointing object 163 0 (offset 0)
Ignori

In [3]:
diretorio = 'arquivos/chat_retrieval_db'

embeddings_model = OpenAIEmbeddings()
vectordb = Chroma.from_documents(
    documents=documents,
    embedding=embeddings_model,
    persist_directory=diretorio
)

In [4]:
from langchain_openai.chat_models import ChatOpenAI

chat = ChatOpenAI(model="gpt-4o-mini")

In [5]:
from langchain.chains.retrieval_qa.base import RetrievalQA

chat_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=vectordb.as_retriever(search_type='mmr'),
)

In [6]:
pergunta = "O que é Hugging Face e como faço para acessá-lo?"
chat_chain.invoke({"query": pergunta})

{'query': 'O que é Hugging Face e como faço para acessá-lo?',
 'result': 'Hugging Face é uma comunidade e plataforma que reúne centenas de milhares de modelos de aprendizado de máquina, especialmente focados em processamento de linguagem natural (NLP). Os modelos disponíveis podem ser usados para uma variedade de tarefas, como geração de texto, resumo e classificação. A comunidade de código aberto da Hugging Face está crescendo rapidamente e busca alcançar o desempenho de modelos proprietários.\n\nPara acessar o Hugging Face, você pode visitar o site oficial da plataforma. Lá, você encontrará uma vasta biblioteca de modelos, tutoriais e documentação para ajudá-lo a começar a usar diferentes modelos para seus projetos. Além disso, você pode explorar mapas de comunidade, fóruns e outros recursos para aprofundar seu conhecimento.'}

### Técnica de Prompt Template

In [7]:
from langchain.prompts import PromptTemplate

chain_prompt = PromptTemplate.from_template(
"""Utilize o contexto fornecido para responder a pergunta ao final. 
Se você não sabe a resposta, apenas diga que não sabe e não invente uma resposta.
Utilize três frases no máximo, mantenha a resposta concisa.

Contexto: {context}

Pergunta: {question}

Resposta:
"""
)

In [8]:
chat_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=vectordb.as_retriever(search_type="mmr"),
    chain_type_kwargs={"prompt":chain_prompt},
    return_source_documents=True
)

In [10]:
pergunta = 'O que é Veleiro Katamarã ?'
#pergunta = 'O que é Hugging Face e como faço para acessá-lo?'
resposta = chat_chain.invoke({'query': pergunta})
print(resposta['result'])

Não sei.


In [11]:
resposta['source_documents']

[Document(metadata={'author': 'lucas', 'total_pages': 28, 'page': 3, 'creationdate': '2016-05-04T10:06:39-03:00', 'source': 'files/apostila.pdf', 'moddate': '2016-05-04T10:06:39-03:00', 'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'page_label': '4', 'doc_id': 9}, page_content='Reforçando, este é um material introdutório. Tem muito mais para aprender em Python: \norientação a objetos, programação funcional, metaprogramação, interface gráfica, \nexpressões regulares, threads, tratamento de exceções, funções anônimas, geradores, \ndesenvolvimento web, aplicativos móveis, entre outras.  \nBem-vindo ao mundo Python! \n \n \n \nProf. Dr. João Luiz Franco \nTutor do grupo PET - ADS / São Carlos'),
 Document(metadata={'doc_id': 51, 'producer': 'macOS Version 13.5 (Build 22G74) Quartz PDFContext', 'creationdate': "D:20231017132635Z00'00'", 'page_label': '1', 'page': 0, 'total_pages': 9, 'creator': 'PyPDF', 'moddate': "D:20231017132635Z00'00'", 'source': 'files/LLM.pdf'

### Debug do LangChain

In [13]:
from langchain.globals import set_debug

set_debug(True)

#pergunta = 'O que é Hugging Face e como faço para acessá-lo?'
pergunta = 'O que é Veleiro Katamarã ?'
resposta = chat_chain.invoke({'query': pergunta})

set_debug(False)

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "O que é Veleiro Katamarã ?"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "O que é Veleiro Katamarã ?",
  "context": "Reforçando, este é um material introdutório. Tem muito mais para aprender em Python: \norientação a objetos, programação funcional, metaprogramação, interface gráfica, \nexpressões regulares, threads, tratamento de exceções, funções anônimas, geradores, \ndesenvolvimento web, aplicativos móveis, entre outras.  \nBem-vindo ao mundo Python! \n \n \n \nProf. Dr. João Luiz Franco \nTutor do grupo PET - ADS / São Carlos\n\nE-BOOK \nUm guia compacto sobre Large Language Models (LLM)\n\n.5, GPT-4 e outros) e geralmente obter uma resposta rápida. Eles estão entre os modelos de maior desempenho, treinados em conjuntos de

### Refinamento da Resposta

In [14]:
chat_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=vectordb.as_retriever(search_type='mmr'),
    chain_type='refine'
)

pergunta = 'O que é Hugging Face e como faço para acessá-lo?'
resposta = chat_chain.invoke({'query': pergunta})
print(resposta['result'])

Hugging Face é uma comunidade e plataforma que fornece acesso a uma ampla gama de modelos de aprendizado de máquina, especialmente voltados para o processamento de linguagem natural (NLP). Esses modelos são extremamente úteis para diversas aplicações, como geração de texto, resumo, tradução, entre outros. A Hugging Face se destaca por sua contribuição ao código aberto e por permitir que desenvolvedores e pesquisadores compartilhem e utilizem modelos colaborativamente.

Recentemente, um dos maiores saltos de desempenho dos modelos da Hugging Face veio da integração do feedback humano diretamente no processo de treinamento. Isso significa que os modelos podem ser refinados com base nas interações e preferências dos usuários, resultando em um desempenho mais alinhado às expectativas e necessidades reais.

Aqui estão alguns passos para começar a usar Hugging Face:

1. **Visitar o site**: Acesse [huggingface.co](https://huggingface.co), onde você encontrará a biblioteca de modelos.

2. **Cr